# NOSQL Data Modeling and ETL pipeline With Apache Cassandra

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [3]:
#pip install cassandra-driver

In [4]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [59]:
# checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/datasets/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to see what the list of event data rows will look like
#print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('./datasets/full_datafile/event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # write the required rows only to the new file
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [62]:
# check the number of rows in your csv file
with open('./datasets/full_datafile/event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Modeling. 

## work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the <font color=red>./datasets/full_datafile directory</font>.
## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [63]:
# connect to cassandra
from cassandra.cluster import Cluster

# setup a cassandra host server using docker
CASSANDRA_HOST = "cassandra"

# conect to cassandra host server
cluster = Cluster([CASSANDRA_HOST])

# establish connection and begin executing queries, init a session
session = cluster.connect()

#### Create Keyspace

In [64]:
# Create a Keyspace
try:
    session.execute("""
                        CREATE KEYSPACE IF NOT EXISTS sparkify
                        WITH REPLICATION = 
                        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
    )
    
except Exception as e:
    print(e)

#### Set Keyspace

In [65]:
# Set KEYSPACE to the keyspace sparkify
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

## Nosql Apache cassandra modeling based on the Analytical quieries

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [66]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = <VALUE>, and itemInSession = <VALUE> 
try:
    session.execute("""
                        CREATE TABLE IF NOT EXISTS songs
                        (
                        session_id INT,
                        item_in_session INT,
                        song_title TEXT,
                        song_length FLOAT,
                        artist TEXT,
                        PRIMARY KEY (session_id, item_in_session)
                        );
    """)

except Exception as e:
    print(e)
        

In [67]:
# load data from the csv to cassandra Nosql db
file = './datasets/full_datafile/event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO songs 
                    (
                    session_id,
                    item_in_session,
                    song_title,
                    song_length,
                    artist
                    ) 
                    VALUES 
                    (%s, %s, %s, %s, %s);
        """
        
        ## execute query
        session.execute(query, (int(line[8]), int(line[3]), line[9], float(line[5]), line[0]))

#### Do a SELECT to verify that the data have been inserted into each table

In [68]:
## SELECT statement to verify the data was entered into the table
query = """
SELECT * 
FROM songs 
WHERE session_id = 338 AND item_in_session = 4;
"""

result = session.execute(query)

for row in result:
    print(row.session_id, row.item_in_session, row.song_title, row.song_length, row.artist)

338 4 Music Matters (Mark Knight Dub) 495.30731201171875 Faithless


In [69]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = <VALUE>, sessionid = <VALUE>;
try:
    session.execute("""
                    CREATE TABLE IF NOT EXISTS users
                    (
                    user_id INT,
                    session_id INT,
                    item_in_session INT,
                    user_firstname TEXT,
                    user_lastname TEXT,
                    song_title TEXT,
                    artist TEXT,
                    PRIMARY KEY (user_id, session_id, item_in_session)
                    );
    """)

except Exception as e:
    print(e)             

In [70]:
# load data from the csv to cassandra Nosql db
file = './datasets/full_datafile/event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO users
                    (
                    user_id,
                    session_id,
                    item_in_session,
                    song_title,
                    user_firstname,
                    user_lastname,
                    artist
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
        """
        ## execute query
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[1], line[2], line[0]))

In [73]:
## SELECT statement to verify the data was entered into the table
query = """
        SELECT * 
        FROM users 
        WHERE user_id = 10 AND session_id = 182;
"""

result = session.execute(query)

for row in result:
    print(row.user_id, row.session_id, row.item_in_session, row.song_title, row.user_firstname, row.user_lastname, row.artist)

10 182 0 Keep On Keepin' On Sylvie F Down To The Bone
10 182 1 Greece 2000 Sylvie F Three Drives
10 182 2 Kilometer Sylvie F Sebastien Tellier
10 182 3 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie F Lonnie Gordon


In [74]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    session.execute("""
                    CREATE TABLE IF NOT EXISTS music_library
                    (
                    user_firstname TEXT,
                    user_lastname TEXT,
                    song_title TEXT,
                    PRIMARY KEY (song_title)
                    );
    """)

except Exception as e:
    print(e)             

                    

In [ ]:
# load data from the csv to cassandra Nosql db
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO music_library 
                    (
                    song_title,
                    user_firstname,
                    user_lastname
                    )
                    VALUES (%s, %s, %s);
        """

        ## execute query
        session.execute(query, (line[9], line[1], line[2]))

In [75]:
## SELECT statement to verify the data was entered into the table
query = """
            SELECT * 
            FROM music_library 
            WHERE song_title = 'All Hands Against His Own';
"""

result = session.execute(query)

for row in result:
    print(row.session_id, row.item_in_session, row.song_title, row.song_length, row.artist)

### Drop the tables before closing out the sessions

In [76]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS songs")
session.execute("DROP TABLE IF EXISTS user")
session.execute("DROP TABLE IF EXISTS music_library")

### Close the session and cluster connection¶

In [77]:
session.shutdown()
cluster.shutdown()